# **Experiment Track with using MLFlow**

In [49]:
from tkinter.scrolledtext import example

from sympy import print_tree

''' Import all important libraries '''
import os
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, random_split, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import pytorch_lightning as pl
from mlflow.models import infer_signature
import matplotlib.pyplot as plt

In [50]:
Device = (torch.device('mps') if torch.mps.is_available() else torch.device('cpu'))
print(Device)

mps


In [51]:
''' Hyperperameters '''
torch.manual_seed(40)
Batch_size = 32
Epochs = 2
Learning_Rate = 0.001

In [52]:
''' Load data and define Source code path '''
Root_path = '/Users/mahadiur/Desktop/Experiment Track Using MLFlow/Data'
dataset_path = os.path.join(Root_path, 'DigitDataset.csv')

saved_model_dir = 'models'
source_code_path = os.path.join(
    os.getcwd(),
    'Experiment_Track_Using_MLFlow.ipynb'
) # current file path

source_code = 'trainer.ipynb'

# **Data Pipeline**

In [66]:
digit_data = pd.read_csv(dataset_path)
example = digit_data.iloc[0]
pixel = example.values[1:]
label = int(example.values[0:1])
print(pixel.shape)
print(label)

(784,)
1


/var/folders/s5/yp49k9zd4457sltd30xmn_2m0000gn/T/ipykernel_3313/1057430082.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(example.values[0:1])


In [54]:
class DataPipeline(Dataset):
    def __init__(self, data_path, transform=None):
        super().__init__()
        self.data = pd.read_csv(data_path)
        self.transformation = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data.iloc[idx]
        pixels = example.values[1:].astype('float32')
        pixels /= 255
        labels = int(example[0:1])

        pixels = torch.tensor(pixels).reshape(28, 28).unsqueeze(0)
        labels = torch.tensor(labels)


        if self.transformation:
            pixels = self.transformation(pixels)

        return  pixels, labels

In [55]:
Transformation = transforms.Compose([
    transforms.Normalize(
        mean=(torch.tensor([0.1307])),
        std=(torch.tensor([0.3081])),
    )
])

In [56]:
dataset = DataPipeline(
    dataset_path,
    Transformation
)
print(len(dataset))

42000


# **Split Train, Test & Validation**

In [57]:
Train_size = int(0.7 * len(dataset))
Validation_size = int(0.15 * len(dataset))
Test_size = len(dataset) - Train_size - Validation_size

Training_dataset, Validation_dataset, Test_dataset = random_split(
    dataset=dataset,
    lengths=[Train_size, Validation_size, Test_size],
)

print(len(Training_dataset))
print(len(Validation_dataset))
print(len(Test_dataset))

29399
6300
6301


# **Dataloader for Train, Test & Validation**

In [58]:
Train_Dataloader = DataLoader(
    dataset=Training_dataset,
    batch_size=Batch_size,
    shuffle=True,

)

Validation_Dataloader = DataLoader(
    dataset=Validation_dataset,
    batch_size=Batch_size,
    shuffle=False,
)

Test_Dataloader = DataLoader(
    dataset=Test_dataset,
    batch_size=Batch_size,
    shuffle=False,

)

In [59]:
for pixels, labels in Train_Dataloader:
    print(pixels.shape)
    print(labels.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


/var/folders/s5/yp49k9zd4457sltd30xmn_2m0000gn/T/ipykernel_3313/1280284542.py:14: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  labels = int(example[0:1])


# **DigitClassifiar class**

In [60]:
class DigitClass(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.criterion = nn.CrossEntropyLoss()
        self.layer1 = nn.Linear(28 * 28, 128)
        self.layer2 = nn.Linear(128, 32)
        self.layer3 = nn.Linear(32, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return x

    # Gradient Decent
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=Learning_Rate)
        return optimizer

    # Training Step
    def training_step(self, batch, batch_idx):
        pixels, labels = batch
        pixels = pixels.to(Device)
        labels = labels.to(Device)
        outputs = self.forward(pixels)
        loss = self.criterion(outputs, labels)
        self.log('Train_loss', loss)
        return loss

    # Validation Step
    def validation_step(self, batch, batch_idx):
        pixels, labels = batch
        pixels = pixels.to(Device)
        labels = labels.to(Device)
        outputs = self.forward(pixels)
        loss = self.criterion(outputs, labels)
        accuracy = (torch.argmax(outputs, dim=1) == labels).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_accuracy', accuracy, prog_bar=True)

    # Test Step
    def test_step(self, batch, batch_idx):
        pixels, labels= batch
        pixels = pixels.to(Device)
        labels = labels.to(Device)
        outputs = self.forward(pixels)
        loss = self.criterion(outputs, labels)
        accuracy = (torch.argmax(outputs, dim=1) == labels).float().mean()
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_accuracy', accuracy, prog_bar=True)



In [61]:
Model = DigitClass().to(Device)

In [62]:
Early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=True,
)

checkpoints_callback = ModelCheckpoint(
    monitor='val_accuracy',
    save_top_k=1,
    mode='max',
)

checkpoints_path = os.path.join(
    os.getcwd(),'checkpoints','Best_Model.pth'
)

# **Train**

In [63]:
Training = pl.Trainer(
    max_epochs=Epochs,
    callbacks=[checkpoints_callback, Early_stopping],
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [64]:
Training.fit(
    model=Model,
    train_dataloaders=Train_Dataloader,
    val_dataloaders=Validation_Dataloader,
)


  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | criterion | CrossEntropyLoss | 0      | train
1 | layer1    | Linear           | 100 K  | train
2 | layer2    | Linear           | 4.1 K  | train
3 | layer3    | Linear           | 330    | train
4 | relu      | ReLU             | 0      | train
-------------------------------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.420     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/var/folders/s5/yp49k9zd4457sltd30xmn_2m0000gn/T/ipykernel_3313/1280284542.py:14: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  labels = int(example[0:1])
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 919/919 [00:04<00:00, 193.03it/s, v_num=1]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 919/919 [00:05<00:00, 160.77it/s, v_num=1, val_loss=0.178, val_accuracy=0.945]

Metric val_loss improved. New best score: 0.178


Epoch 1: 100%|██████████| 919/919 [00:04<00:00, 193.85it/s, v_num=1, val_loss=0.178, val_accuracy=0.945]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 919/919 [00:05<00:00, 162.00it/s, v_num=1, val_loss=0.148, val_accuracy=0.956]

Metric val_loss improved by 0.030 >= min_delta = 0.0. New best score: 0.148
`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 919/919 [00:05<00:00, 161.74it/s, v_num=1, val_loss=0.148, val_accuracy=0.956]
